In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/biubug6/Pytorch_Retinaface.git

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py --network mobile0.25 --training_dataset 'path_to_train_labels'

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

import torch
from models.retinaface import RetinaFace
from data import cfg_mnet, cfg_re50

from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
from utils.box_utils import decode, decode_landm

In [ ]:
weight_path = 'path_to_weight_file'
cfg = cfg_mnet # mobile0.25 (cfg_mnet) or resnet50 (cfg_re50)
resize = 1
confidence_threshold = 0.02
top_k = 5000
nms_threshold = 0.4
keep_top_k = 750
vis_thres = 0.6

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
model = RetinaFace(cfg, phase = 'test').to(device)
model.load_state_dict(torch.load(weight_path, map_location=device))
model.eval()
print("Model Loaded!")

In [ ]:
def retinaface_inf(test_img, model):
    # Convert image to numpy float32 format
    img = np.float32(test_img)
    im_height, im_width, _ = img.shape

    # Define scale for image
    scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
    # Subtract mean values from image
    img -= (104, 117, 123)
    # Transpose image to required format
    img = img.transpose(2, 0, 1)
    # Convert numpy array to torch tensor and add batch dimension
    img = torch.from_numpy(img).unsqueeze(0)
    # Move image and scale to device
    img = img.to(device)
    scale = scale.to(device)

    tic = time.time()
    # Perform forward pass through the model
    loc, conf, landms = model(img)

    # Calculate prior boxes
    priorbox = PriorBox(cfg, image_size=(im_height, im_width))
    priors = priorbox.forward()
    priors = priors.to(device)
    prior_data = priors.data
    # Decode predicted locations to bounding boxes
    boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
    boxes = boxes * scale / resize
    boxes = boxes.cpu().numpy()
    # Extract confidence scores
    scores = conf.squeeze(0).data.cpu().numpy()[:, 1]

    # Ignore low confidence detections
    inds = np.where(scores > confidence_threshold)[0]
    boxes = boxes[inds]
    scores = scores[inds]

    # Keep top-K detections before Non-Maximum Suppression (NMS)
    order = scores.argsort()[::-1][:top_k]
    boxes = boxes[order]
    scores = scores[order]

    # Perform NMS
    dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
    keep = py_cpu_nms(dets, nms_threshold)
    dets = dets[keep, :]

    # Keep top-K detections after NMS
    dets = dets[:keep_top_k, :]

    # Calculate frames per second (FPS)
    fps_ = round(1/(time.time() - tic), 2)
    # Draw bounding boxes on the image
    for b in dets:
        if b[4] < vis_thres:
            continue
        b = list(map(int, b))
        cv2.rectangle(test_img, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 4)

    # Define font properties
    font_scale = 2
    thickness = 3
    font = cv2.FONT_HERSHEY_SIMPLEX
    # Add label "Retinaface" to the image
    text = "Retinaface"
    (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_position = (im_width - text_width - 10, im_height - 20)
    cv2.putText(test_img, text, text_position, font, font_scale, (0, 0, 255), thickness, cv2.LINE_AA)

    # Add FPS information to the image
    fps_text = "FPS: " + str(fps_)
    (text_width, text_height), _ = cv2.getTextSize(fps_text, font, font_scale, thickness)
    fps_position = (im_width - text_width - 10, im_height - 20 - text_height - 10)
    cv2.putText(test_img, fps_text, fps_position, font, font_scale, (0, 0, 255), thickness, cv2.LINE_AA)

    return test_img


In [ ]:
test_path = 'path_to_test_image'
test_img = cv2.imread(test_path)

retina_img = retinaface_inf(test_img, model)

plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
import dlib
# Initialize dlib's face detector
face_detector = dlib.get_frontal_face_detector()

In [ ]:
def dlib_inf(test_img, face_detector):
    # Convert image to numpy float32 format
    img = np.float32(test_img)
    im_height, im_width, _ = img.shape

    tic = time.time()
    # Perform face detection using dlib
    face_detection = face_detector(test_img)
    fps_ = round(1/(time.time() - tic), 2)
    # Draw bounding boxes on the image
    for f in face_detection:
        cv2.rectangle(test_img, (f.left(), f.top()), (f.right(), f.bottom()), (255,0,0), 4)
    # Define font properties
    font_scale = 2
    thickness = 3
    font = cv2.FONT_HERSHEY_SIMPLEX
    # Add label "Dlib" to the image
    text = "Dlib"
    (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)
    text_position = (im_width - text_width - 10, im_height - 20)
    cv2.putText(test_img, text, text_position, font, font_scale, (255, 0, 0), thickness, cv2.LINE_AA)

    # Add FPS information to the image
    fps_text = "FPS: " + str(fps_)
    (text_width, text_height), _ = cv2.getTextSize(fps_text, font, font_scale, thickness)
    fps_position = (im_width - text_width - 10, im_height - 20 - text_height - 10)
    cv2.putText(test_img, fps_text, fps_position, font, font_scale, (255, 0, 0), thickness, cv2.LINE_AA)

    return test_img

In [ ]:
test_path ='path_to_test_image'
test_img = cv2.imread(test_path)

dlib_img = dlib_inf(test_img, face_detector)

plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
plt.show()